In [1]:
import math
import pyaudio
import itertools
import numpy as np
import seaborn as sns
from pygame import midi
from scipy.io import wavfile
import matplotlib.pyplot as plt
from IPython.display import Audio

pygame 2.6.1 (SDL 2.28.4, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from synth.components.envelopes import ADSREnvelope
from synth.components.composers import WaveAdder, Chain
from synth.components.modifiers import Panner, ModulatedPanner, ModulatedVolume
from synth.components.oscillators import ModulatedOscillator, SawtoothOscillator
from synth.components.oscillators import TriangleOscillator, SineOscillator, SquareOscillator

## MIDI

In [3]:
sns.set_theme()
SR = 44_100 # Sample rate

figsize=(25, 6.25)
colors = "#323031", "#308E91", "#34369D","#5E2A7E", "#5E2A7E", "#6F3384"

In [4]:
getfig = lambda : plt.figure(figsize=figsize)
savefig = lambda fig, name: fig.savefig(f"./tempimg/{name}.jpg")
def plot(xy, r=1,c=1,i=1,title="", xlabel="",ylabel="",yticks=None, xticks=None,**plot_kwargs):
    # plt.plot helper
    if r > 0:
        plt.subplot(r,c,i)
    plt.title(title)
    if len(xy) == 2:
        plt.plot(*xy, **plot_kwargs)
    else:
        plt.plot(xy, **plot_kwargs)
        
    if xticks is not None: plt.xticks(xticks)
    if yticks is not None: plt.yticks(yticks)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)

In [11]:
!pip install mido python-rtmidi


In [12]:
import mido

print("Puertos de entrada MIDI disponibles:")
for name in mido.get_input_names():
    print("   ", name)


Puertos de entrada MIDI disponibles:
    Launchkey Mini MK4 37 MIDI 0
    MIDIIN2 (Launchkey Mini MK4 37  1


In [16]:
import time

port_name = "Launchkey Mini MK4 37 MIDI 0"  # ← reemplaza con el nombre exacto
with mido.open_input(port_name) as inport:
    print(f"Escuchando «{port_name}» 5 s…")
    t0 = time.time()
    while time.time() - t0 < 5:
        msg = inport.receive(block=False)
        if msg:
            print(msg)
        time.sleep(0.01)
    print("Fin de la escucha.")


Escuchando «Launchkey Mini MK4 37 MIDI 0» 5 s…
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
note_on channel=13 note=61 velocity=76 time=0
clock time=0
clock time=0
note_on channel=13 note=62 velocity=44 time=0
clock time=0
clock time=0
note_on channel=13 note=61 velocity=0 time=0
clock time=0
note_on channel=13 note=62 velocity=0 time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
note_on channel=13 note=62 velocity=62 time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
note_on channel=13 note=62 velocity=0 time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
clock time=0
note

I found this [link](https://users.cs.cf.ac.uk/Dave.Marshall/Multimedia/node158.html) to be useful in decoding the `status` codes.

## Super Simple Synth

Now that we have MIDI kinda out of the way we can create a synth using the most basic oscillator. In the [first post on oscillators](http://localhost:8888/notebooks/synth/Post%20Oscillators.ipynb) we had coded a simple sine wave generator. We can add an additional argument for `amp` to scale the amplitude of the note.

In [17]:
def get_sin_oscillator(freq=55, amp=1, sample_rate=44100):
    increment = (2 * math.pi * freq)/ sample_rate
    return (math.sin(v) * amp for v in itertools.count(start=0, step=increment))

For the super simple synth we'll use the oscillators from the above function.

Before we get to the playing part we need to route the synthesizer output to the speaker, for this we can use [`pyaudio`](http://people.csail.mit.edu/hubert/pyaudio/). We need to first set up a [PyAudio stream](http://people.csail.mit.edu/hubert/pyaudio/docs/#class-stream) to which we can write the synth values.

In [ ]:
stream = pyaudio.PyAudio().open(
    rate=44100,
    channels=1,
    format=pyaudio.paInt16,
    output=True,
    frames_per_buffer=256
)

The parameters of the stream object:
- `rate` is the sample rate of the audio.
- `channels` is the number of audio channels (left and right).
- `format` is the format of each sample which here is set to signed 16 bit integers cause there is no option for floats.
- `output` is set to `True` cause we are "writing" to a speaker rather than "reading" from some input.
- `frames_per_buffer` tells the stream object how many samples from the synthesizer we'll be feeding it at a time.

Now that we have our stream object we just need to call the `stream.write` method to get it to play our synth sounds. We just need another function to get the required number of samples from an oscillator in the correct format, which here is int 16 which means that the numbers range from -2^15 to 2^15 - 1, i.e. (-32768, 32767).

In [19]:
def get_samples(osc, num_samples=256):
    return [int(next(osc) * 32767) for i in range(256)]

In [ ]:
fig = getfig()
osc = get_sin_oscillator(freq=55)
n = 5
for i in range(1,n+1):
    plot(get_samples(osc), 1,n,i, color=colors[3], ylabel="value" if i == 1 else "", xlabel="samples")
    plt.title(f"Buffer State {i}")
    
savefig(fig, "5buffers")

This is how we'll "play" the synth:
- Note is pressed
    - Oscillator of the correct frequency is created and added to a `dict`.
    - Values from all the in the `dict` are added and returned in a frame of a given size.
    - Values in the frame are written to `stream`.
- Note is released
    - Oscillator is removed from the `dict`.

We'll be using a `dict` to keep track of all the notes that are pressed down. The key will be the midi note value and item will be the oscillator. We can get the frequency of the note using the `midi.midi_to_frequency` function.

Since we are using a `dict` of oscillators to play the notes from, we'll have to rewrite the `get_samples` function.

In [21]:
def get_samples(notes_dict, num_samples=256):
    return [sum(int(next(osc) * 32767)
                for osc in notes_dict.values())
            for _ in range(num_samples)]

Basically what's going on in the above list comprehension is: get the sum of the values of all the oscillators in 16 bit integer format `num_samples` times, by calling `next` on the oscillators to get their each of their values.

Since we have to write to stream in `bytes` format we have to use `np.int16(samples).tobytes()`. Now if you run the above code you have a polyphonic sine wave synthesizer. 

In [23]:
# Diccionario de osciladores activos
notes_dict = {}
try:
    print("Toca unas notas — Ctrl+C para parar")
    while True:
        # 1) Generar audio si hay voces activas
        if notes_dict:
            samples = get_samples(notes_dict)
            stream.write(np.int16(samples).tobytes())

        # 2) Leer todos los mensajes pendientes con Mido
        for msg in inport.iter_pending():
            # msg es un objeto mido.Message
            if msg.type == 'note_on' and msg.velocity > 0:
                # arrancar nota
                freq = 440.0 * 2**((msg.note - 69)/12.0)
                osc = get_sin_oscillator(freq=freq, amp=msg.velocity/127.0)
                notes_dict[msg.note] = osc
            elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                # terminar nota
                notes_dict.pop(msg.note, None)

        time.sleep(0.005)

except KeyboardInterrupt:
    print("Stopping...")

Toca unas notas — Ctrl+C para parar
Stopping...


- In this file [here](link_to_the_file.com), I have put together all of the above code, so you can use `$ python super_simple_synth.py` and if all your libraries and MIDI controller are in place the synth should be running you can quit by pressing `control + c`.
- I got a bit carried away while making coding the Super Simple Synth, so [here's](link_to_the_other_file.com) the same thing in 19 lines and 676 characters.

In [ ]:
Audio("tempsnd/something_familiar_deux.wav")

## The Synthesizer

This is how it sounds, the a very raw synth, no modulations, pure sine. Notice the clicks at the start and end of a note, this is because there is no ADSR applied to the amplitude.

In [ ]:
def get_sin_oscillator(freq, amp, sample_rate):
    increment = (2 * math.pi * freq)/ sample_rate
    return (math.sin(v) * amp for v in itertools.count(start=0, step=increment))

class PolySynth:
    def __init__(self, amp_scale=0.3, max_amp=0.8, sample_rate=44100, num_samples=64):
        # Initialize MIDI
        midi.init()
        if midi.get_count() > 0:
            self.midi_input = midi.Input(midi.get_default_input_id())
        else: 
            raise Exception("no midi devices detected")
        
        # Constants
        self.num_samples = num_samples
        self.sample_rate = sample_rate
        self.amp_scale = amp_scale
        self.max_amp = max_amp
    
    def _init_stream(self, nchannels):
        # Initialize the Stream object
        self.stream = pyaudio.PyAudio().open(
            rate=self.sample_rate,
            channels=nchannels,
            format=pyaudio.paInt16,
            output=True,
            frames_per_buffer=self.num_samples
        )
        
    def _get_samples(self, notes_dict):
        # Return samples in int16 format
        samples = []
        for _ in range(self.num_samples):
            samples.append(
                [next(osc[0]) for _, osc in notes_dict.items()]
            )
        samples = np.array(samples).sum(axis=1) * self.amp_scale
        samples = np.int16(samples.clip(-self.max_amp, self.max_amp) * 32767)
        return samples.reshape(self.num_samples, -1)

    def play(self, osc_function=get_sin_oscillator, close=False):
        tempcf = osc_function(1, 1, self.sample_rate)
        has_trigger = hasattr(tempcf, "trigger_release")
        tempsm = self._get_samples({-1: [tempcf, False]})
        nchannels = tempsm.shape[1]
        self._init_stream(nchannels)

        try:
            notes_dict = {}
            while True:
                if notes_dict:
                    # Play the notes
                    samples = self._get_samples(notes_dict)
                    self.stream.write(samples.tobytes())
                    
                if self.midi_input.poll():
                    # Add or remove notes from notes_dict
                    for event in self.midi_input.read(num_events=16):
                        (status, note, vel, _), _ = event
                        if status == 0x80 and note in notes_dict:
                            if has_trigger:
                                notes_dict[note][0].trigger_release()
                                notes_dict[note][1] = True
                            else:
                                del notes_dict[note]

                        elif status == 0x90:
                            freq = midi.midi_to_frequency(note)
                            notes_dict[note] = [
                                osc_function(freq=freq, amp=vel/127, sample_rate=self.sample_rate), 
                                False
                            ]

                if has_trigger:
                    # Delete notes if ended
                    ended_notes = [k for k,o in notes_dict.items() if o[0].ended and o[1]]
                    for note in ended_notes:
                        del notes_dict[note]

        except KeyboardInterrupt as err:
            self.stream.close()
            if close:
                self.midi_input.close()

#### rec_play

In [ ]:
def rec_play(self, osc_function=get_sin_oscillator, close=False, file_name=None):
    rec = []
    tempcf = osc_function(1, 1, self.sample_rate)
    has_trigger = hasattr(tempcf, "trigger_release")
    tempsm = self._get_samples({-1: [tempcf, False]})
    nchannels = tempsm.shape[1]
    self._init_stream(nchannels)

    try:
        notes_dict = {}
        while True:
            if notes_dict:
                # Play the notes
                samples = self._get_samples(notes_dict)
                self.stream.write(samples.tobytes())
                if file_name:
                    rec.append(samples)

            if self.midi_input.poll():
                # Add or remove notes from notes_dict
                for event in self.midi_input.read(num_events=16):
                    (status, note, vel, _), _ = event
                    if status == 0x80 and note in notes_dict:
                        if has_trigger:
                            notes_dict[note][0].trigger_release()
                            notes_dict[note][1] = True
                        else:
                            del notes_dict[note]

                    elif status == 0x90:
                        freq = midi.midi_to_frequency(note)
                        notes_dict[note] = [
                            osc_function(freq=freq, amp=vel/127, sample_rate=self.sample_rate), 
                            False
                        ]

            if has_trigger:
                # Delete notes if ended
                ended_notes = [k for k,o in notes_dict.items() if o[0].ended and o[1]]
                for note in ended_notes:
                    del notes_dict[note]

    except KeyboardInterrupt as err:
        if file_name:
            wavfile.write(file_name, self.sample_rate, np.concatenate(rec))
            self.stream.close()
        if close:
            self.midi_input.close()

#### Run

#### Content

We'll wrap up all the above code in a class, cause classes generally make things easier to deal with. The `PolySynth` class below is an example of how we could use the components to create a synthesizer.

The `PolySynth` class will help us play the synth using a MIDI controller, i.e. it will let us play some notes on some input device, the notes will be converted to a sequence of numbers which will then be routed to the speaker for playback.

### PolySynth Methods

**`__init__`** 
- The object initialization function `__init__` takes two new arguments:
    1. `amp_scale` : this scales the value of a sample, this is used to reduce the volume of a single note cause during playback when multiple notes are played their values are added unlike the `WaveAdder` where the values are averaged.
    2. `max_amp` : this is used to clip all the values to a range of `[-max_amp, max_amp]` this is so that we don't damage our speakers if the volume gets too loud. *Note : If you set this value to be sufficiently low (example 0.01) it will create a distortion effect.*
- Both the above parameters are for speaker safety, other then these two it will also take in `frames_per_buffer` and `sample_rate` these are the same as the ones explained in the earlier section.
- When a `Player` instance is created, it initializes the `midi.Input` object and saves the passed arguments.

**`_init_stream`**
- This function is used to initialize the stream object before a synth is played, this is set as a separate function and not initialized in `__init__` cause the number of channels are decided by the type of oscillator being used.

**`_get_samples`**
- This function will basically call `next` on each of the oscillators in the `notes_dict` sufficient number of times to fill the buffer which will be fed to the output (speaker).
- The output of each of the oscillators is summed up, then scaled using `amp_scale` then it's clipped to `max_amp` and finally it is converted to 16 bit integers before returning.
- Since the oscillators can return 2 samples if it's stereo we use `numpy` for the quicker operations.
- The `notes_dict` values are a list of two items: `[0]` is the oscillator and `[1]` is a boolean flag that indicates if a note can be deleted.

**`play`**
- This is the main function of the class and is what will help us play the synthesizer and record what we play to a `.wav` file.
- First I'll go over the arguments:
    - `osc_function` : This is any function that will return an oscillator depending on the midi input, more on this in a while.
    - `close` : If this is set to `True` then it will close the midi input object.
- The general structure of the `play` function is the same as the loop describe earlier, the major differences are
    - It allows for *generators* that have release triggers (using `trigger_release`) on them such as `ModifiedOscillators` using an `ADSREnvelope`.
    - If a release trigger is present it doesn't delete a note immediately, only when the note has `ended` and the delete flag in `notes_dict` has been set.

### `osc_function`

- The `osc_function`, this is a function that has the following signature
    ```python
    osc_function(freq:float, amp:float, sample_rate:int) -> Iterable
    ```
    when a midi note is played the frequency and amplitude of the note is passed to the `osc_function` which will then return an oscillator (`osc`) that on calling `next` on it (`next(osc)`) will return the value to be played.
- An example of the `osc_function` is the `get_sin_oscillator` function described in one of the earlier sections, this is the simplest `osc_function`.
- Using the components defined in part one and two of this series we can make more complex `osc_function`s such as
    ```python
    def osc_function(freq, amp, sample_rate):
        return iter(
            Chain(
                TriangleOscillator(freq=freq, amp=amp, sample_rate=sample_rate),
                ModulatedPanner(
                    SineOscillator(freq/100, phase=90, sample_rate=sample_rate)
                ),
                ModulatedVolume(ADSREnvelope(0.01, release_duration=0.001, sample_rate=sample_rate))
            )
        )
    
    ```
    because of the design of the previous components we have to call `__iter__` on it to initialize the underlying components such as the `Oscillator`; the above designed `osc_function` sets the `ModulatedPanner`'s oscillator as a function of note frequency, this is what it sounds like. `# embed fmod_pan.wav`

In [ ]:
def osc_function(freq, amp, sample_rate):
    return iter(
        Chain(
            TriangleOscillator(freq=freq, 
                    amp=amp, sample_rate=sample_rate),
            ModulatedPanner(
                SineOscillator(freq/100, 
                    phase=90, sample_rate=sample_rate)
            ),
            ModulatedVolume(
                ADSREnvelope(0.01, 
                    release_duration=0.001, sample_rate=sample_rate)
            )
        )
    )

rec_play(player, osc_function, file_name="tempsnd/fmod_pan.wav")
Audio("tempsnd/fmod_pan.wav")

- One caveat about the above design is that it doesn't limit the number of notes being played so if the oscillators can't generate notes fast enough your output will be kinda wonky, timing maybe off if you are recording from the buffer or it may distort.

In [ ]:
def osc_function(freq, amp, sample_rate):
    return iter(
        Chain(
            WaveAdder(
                Chain(
                    SineOscillator(freq=freq+4, amp=amp, sample_rate=sample_rate),
                    Panner(0.3)
                ),
                Chain(
                    TriangleOscillator(freq=freq, amp=amp, sample_rate=sample_rate),
                    Panner(0.7)
                ),
                Chain(
                    SawtoothOscillator(freq=freq/2, amp=amp*0.1, sample_rate=sample_rate),
                    Panner()
                )
            ),
            ModulatedVolume(ADSREnvelope(0.01,0.2,0.9,0.001))
        )
    )

player.play(osc_function)

In [ ]:
Audio("tempsnd/Osc Struggle.wav")

In [ ]:
def get_glider(old_freq,new_freq, glide, sample_rate):
    np.linspace(old_freq, new_freq, int(sample_rate * glide))
    if old_freq == new_freq:
        glider = itertools.cycle([new_freq])
    else: 
        glider = itertools.count(old_freq, (new_freq-old_freq)/(glide*sample_rate))
    for f in glider:
        if old_freq > new_freq:
            yield max(f, new_freq)
        else:
            yield min(f, new_freq)
        
class MonoSynth(PolySynth):
    def _get_samples(self, osc, glider):
        samples = []
        for _ in range(self.num_samples):
            freq = next(glider)
            if freq != osc.freq:osc.freq = freq
            samples.append(next(osc))
        samples = (np.array(samples) * self.amp_scale).clip(-self.max_amp, self.max_amp)
        return np.int16(samples * 32767)
        
    def play(self, osc, close=False, glide=0.1):
        self._init_stream(1)

        try:
            play = False
            note = None
            glider = None
            
            while True:
                if play:
                    # Play the notes
                    samples = self._get_samples(osc, glider)
                    
                    # Stream Samples
                    self.stream.write(samples.tobytes())
                    
                if self.midi_input.poll():
                    # Add or remove notes from notes_dict
                    for event in self.midi_input.read(num_events=16):
                        (status, note_, vel, _), _ = event
                        freq = midi.midi_to_frequency(note_)
                        
                        if status == 0x80 and note == note_:
                            play = False
                            
                        elif status == 0x90:
                            if not play:
                                play = True
                                osc.freq = freq
                            glider = get_glider(osc.freq, freq, glide, self.sample_rate)
                            note = note_

        except KeyboardInterrupt as err:
            self.stream.close()
            if close:
                self.midi_input.close()

- This means that you can't use very fancy oscillator setups, for that to work properly you'd have to use more efficient code, such as where the samples are generated in required batch sizes rather than one by one and all operations are vectorized, or a faster (perhaps compiled) language; this is why people say Python is slow, it can't perform compile time optimization cause the [reference implementation](https://github.com/python/cpython) is interpreted, but I can't think of any other language that will let me code a polysynth in fewer than 20 lines which is why it's awesome.

- The player class is just one way to go about creating a synth you can have other designs using the earlier coded components, example: only one note plays at a time using just one oscillator by setting it's `._f` parameter.

In [ ]:
player = MonoSynth()

In [ ]:
player.play(iter(SineOscillator()), glide=0.01)

In [ ]:
# player = MonoSynth()
player.play(iter(SineOscillator()), glide=0.2)

In [ ]:
def osc_function(freq, amp, sample_rate):
    return iter(
        Chain(
            ModulatedOscillator(
                TriangleOscillator(freq=freq, amp=amp, sample_rate=sample_rate),
                ADSREnvelope(0.04, decay_duration=0.01, release_duration=0.01),
                amp_mod=lambda x,y:x*y
            ),
            Panner(max(min(freq / 261 - 0.5,0.85),0.15))
        )
    )
player.play(osc_function)

- Here's a `MonoSynth` class that does exactly that by overriding the `PolySynth`'s `play` and `_get_samples` functions.
- Instead of playing the next frequency immediately it glides to the new notes frequency, the duration is decided by the `glide` argument.
- The `get_glider` function basically uses `np.linspace` to interpolate between the old and the new frequencies frequencies.
- This is what it sounds like.

In [ ]:
Audio("tempsnd/For Elise.wav")

This was possible by making setting the `Panner` parameter `r` as function of `freq`.

And that is basically how one would go about making a synth in python, a few things could be done better to increase the number of concurrent oscillators but since we aren't making a VST that we can plug into our DAW it's cool, if you do want to do that you'd have to use C++ or some other compiled language.
Anyway here's Für Elise, in stereoscopic polyphonic glory:

In [ ]:
Audio("tempsnd/Long Glide.wav")

And if you increse the `glide` duration then it sounds very much like a theremin.

In [ ]:
Audio("tempsnd/something_familiar_trois.wav")